In [1]:
import pandas as pd

In [2]:
# Import data
df = pd.read_csv('311_Service_Requests_from_2020_to_Present_20260220.csv')

In [3]:
# Rename columns for better readability
df = df.rename(columns={
    "count_complaint_type": "count",
    "Problem (formerly Complaint Type)": "complaint_type",
    "Agency": "agency",
    "Borough": "borough"
})

In [4]:
df.columns

Index(['count', 'complaint_type', 'agency', 'borough'], dtype='object')

In [5]:
# Remove commas from 'count' column and convert to integer type
df["count"] = (
    df["count"]
      .str.replace(",", "", regex=False)
      .astype(int)
)

# Convert 'count' column to integer type
df["count"] = df["count"].astype(int)

# Standardize 'borough' names to title case
df["borough"] = df["borough"].str.title()

# Remove leading and trailing whitespace from 'complaint_type' column
df["complaint_type"] = df["complaint_type"].str.strip()

# Remove leading and trailing whitespace from 'agency' column
df["agency"] = df["agency"].str.strip()

In [6]:
# Display the top 15 most common complaint types
top_complaints = (
    df.groupby("complaint_type")["count"]
      .sum()
      .sort_values(ascending=False)
)

top_complaints.head(10)

complaint_type
Illegal Parking            577257
Noise - Residential        463349
HEAT/HOT WATER             315946
Noise - Street/Sidewalk    173049
Blocked Driveway           172723
UNSANITARY CONDITION       117720
Water System                77517
Street Condition            70330
PLUMBING                    69535
Abandoned Vehicle           67903
Name: count, dtype: int64

In [7]:
# Get the top 10 complaint types
top10 = top_complaints.head(10).index.tolist()

In [8]:
# Filter the DataFrame to include only the top 10 complaint types
df_top10 = df[df["complaint_type"].isin(top10)]

In [9]:
df_top10

,count,complaint_type,agency,borough
5,9676,Abandoned Vehicle,NYPD,Bronx
6,19981,Abandoned Vehicle,NYPD,Brooklyn
7,3009,Abandoned Vehicle,NYPD,Manhattan
8,29977,Abandoned Vehicle,NYPD,Queens
9,5250,Abandoned Vehicle,NYPD,Staten Island
10,10,Abandoned Vehicle,NYPD,Unspecified
82,27113,Blocked Driveway,NYPD,Bronx
83,61840,Blocked Driveway,NYPD,Brooklyn
84,4394,Blocked Driveway,NYPD,Manhattan
85,74217,Blocked Driveway,NYPD,Queens


In [10]:
# Remove rows where 'borough' is 'Unspecified'
df_top10 = df_top10[df_top10["borough"] != "Unspecified"]

In [11]:
# Group by 'borough' and 'complaint_type' to get the total count for each combination
borough_type = (
    df_top10.groupby(["borough", "complaint_type"])["count"]
      .sum()
      .reset_index()
)

In [12]:
# Group by 'complaint_type' and 'agency' to get the total count for each combination
type_agency = (
    df_top10.groupby(["complaint_type", "agency"])["count"]
      .sum()
      .reset_index()
)

## Structure before Sankey

### Step 1. Create a node list (ordered properly)

In [13]:
# Define the list of boroughs in New York City
boroughs = ["Bronx", "Brooklyn", "Manhattan", "Queens", "Staten Island"]

In [14]:
# Get the order of complaint types based on total count
complaint_order = (
    df_top10.groupby("complaint_type")["count"]
      .sum()
      .sort_values(ascending=False)
      .index
      .tolist()
)

In [15]:
# Get the unique agencies from the 'type_agency' DataFrame
agencies = type_agency["agency"].unique().tolist()

In [16]:
# Combine all nodes (boroughs, complaint types, and agencies) into a single list
nodes = boroughs + complaint_order + agencies

In [17]:
# Create a mapping of node names to their corresponding indices
node_index = {name: i for i, name in enumerate(nodes)}

### Step 2: Build Source, Target, Value Lists

In [18]:
# Initialize empty lists to store the source indices, target indices, and values for the Sankey diagram
sources = []
targets = []
values = []

In [19]:
# Populate the source, target, and value lists for the borough to complaint type connections
for _, row in borough_type.iterrows():
    sources.append(node_index[row["borough"]])
    targets.append(node_index[row["complaint_type"]])
    values.append(row["count"])

In [20]:
# Populate the source, target, and value lists for the complaint type to agency connections
for _, row in type_agency.iterrows():
    sources.append(node_index[row["complaint_type"]])
    targets.append(node_index[row["agency"]])
    values.append(row["count"])

### Step 3: Borough-Colored Flows

In [21]:
# Define colors for each borough
borough_colors = {
    "Bronx": "#1f77b4",
    "Brooklyn": "#ff7f0e",
    "Manhattan": "#2ca02c",
    "Queens": "#d62728",
    "Staten Island": "#9467bd"
}

In [22]:
# Create a list of colors for the links in the Sankey diagram
link_colors = []

# Borough → Complaint colors
for _, row in borough_type.iterrows():
    link_colors.append(borough_colors[row["borough"]])

# Complaint → Agency colors
for _ in range(len(type_agency)):
    link_colors.append("rgba(150,150,150,0.4)")

### Step 4: Build Plotly Sankey

In [28]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        pad=15,
        thickness=15,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    )
)])

fig.update_layout(
    title_text="How 311 Complaints Flow Through NYC Government (2025)",
    font_size=12
)

fig.show()

In [77]:
# Create the Sankey diagram using Plotly
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        pad=15,
        thickness=15,
        line=dict(color="white", width=0.5),
        label=nodes
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors,
        # slightly transparent on hover for clarity
        hoverlabel=dict(font=dict(size=18, family="Silom"))
    )
)])

fig.update_layout(
    title_text="Borough by Borough: Where do NYC's 311 Complaints Go?",
    title_font=dict(size=28, family="Silom", color="#333"),
    font_size=12,
    hovermode="x",  # improve hover interactions
    font=dict(size=14, color="black", family="Silom"),
)

fig.show()

In [ ]:
# Save the Sankey diagram as an HTML file
fig.write_html("nyc_311_sankey_2025.html")

# Optionally customize the output file by adding CSS or JavaScript later